In [1]:
from preprocessing import get_LSTM_dfs, get_features_and_target
import tensorflow as tf

In [2]:
symbol = "BTC/USDT"
feature_lags = []

df = get_features_and_target(
    symbol, day_to_forecast=1, feature_lags=feature_lags, model="LSTM"
)

train_df, valid_df = get_LSTM_dfs(valid_split=2000, feature_lags=[])

for X_batch, y_batch in train_df.take(1):
    print("Train batch X shape:", X_batch.shape)
    print("Train batch y shape:", y_batch.shape)  # starts from y.iloc[48]

Train batch X shape: (32, 48, 15)
Train batch y shape: (32,)


2023-11-19 23:27:14.324963: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-11-19 23:27:14.324985: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-19 23:27:14.324989: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-19 23:27:14.325016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-19 23:27:14.325030: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
tf.keras.backend.clear_session()

conv1 = tf.keras.layers.Conv1D(
    filters=60,
    kernel_size=3,
    strides=1,
    padding="causal",
    activation="relu",
    input_shape=[None, len(df.columns) - 1],
)

# Bidirectional LSTM tf.keras.layers
lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(30, return_sequences=True))
lstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(30, return_sequences=True))

# Model construction
inputs = tf.keras.layers.Input(shape=(None, len(df.columns) - 1))
x = conv1(inputs)
x = lstm1(x)
x = lstm2(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(60, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1)(x)

# SGD optimizer and Huber loss
optimizer = tf.keras.optimizers.legacy.SGD(
    learning_rate=0.001, momentum=0.9, nesterov=True
)
adam_opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

huber_loss = tf.keras.losses.Huber()
mse_loss = "mse"

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=adam_opt, loss=mse_loss, metrics=["mae", "RootMeanSquaredError"]
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 15)]        0         
                                                                 
 conv1d (Conv1D)             (None, None, 60)          2760      
                                                                 
 bidirectional (Bidirection  (None, None, 60)          21840     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, None, 60)          21840     
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, None, 60)          0         
                                                                 
 dense (Dense)               (None, None, 60)          3660  

In [6]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, monitor="val_mae", restore_best_weights=True
)
tensorboard = tf.keras.callbacks.TensorBoard("tensorboard_logs")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_mae",
    factor=0.5,  # Reduce learning rate by half
    patience=2,  # Number of epochs with no improvement
    min_lr=0.0001,  # Minimum learning rate
)
model.fit(
    train_df,
    validation_data=valid_df,
    callbacks=[early_stopping, tensorboard, reduce_lr],
    epochs=50,
)

Epoch 1/50


2023-11-19 23:30:20.554017: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1540/1540 [==============================] - 98s 60ms/step - loss: 0.0022 - mae: 0.0300 - root_mean_squared_error: 0.0464 - val_loss: 4.8725e-04 - val_mae: 0.0134 - val_root_mean_squared_error: 0.0221 - lr: 0.0010
Epoch 2/50
1540/1540 [==============================] - 93s 61ms/step - loss: 0.0017 - mae: 0.0274 - root_mean_squared_error: 0.0411 - val_loss: 4.8643e-04 - val_mae: 0.0134 - val_root_mean_squared_error: 0.0221 - lr: 0.0010
Epoch 3/50
1540/1540 [==============================] - 93s 60ms/step - loss: 0.0017 - mae: 0.0274 - root_mean_squared_error: 0.0411 - val_loss: 4.8624e-04 - val_mae: 0.0134 - val_root_mean_squared_error: 0.0221 - lr: 0.0010
Epoch 4/50
1540/1540 [==============================] - 92s 60ms/step - loss: 0.0017 - mae: 0.0273 - root_mean_squared_error: 0.0410 - val_loss: 4.8448e-04 - val_mae: 0.0137 - val_root_mean_squared_error: 0.0220 - lr: 5.0000e-04
Epoch 5/50
1540/1540 [==============================] - 92s 60ms/step - loss: 0.0017 - mae: 0.0273 - root_m

In [7]:
model.save("last_model.h5")  # Save the model

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
